# NO PCA and no NaN imputing

## Train-Test Split

In [ ]:
from src.utils.filtering import filter_data
from src.utils.data_loading import load_data
from sklearn.model_selection import train_test_split
from src.utils.label_encoding import label_encode_column
from sklearn.preprocessing import StandardScaler

In [ ]:
selected_features = [
    "geocentric_latitude",  # Latitude of conjunction point [deg]
    "c_sigma_rdot",  # covariance; radial velocity standard deviation (sigma) of chaser [m/s]
    "c_obs_used",  # number of observations used for orbit determination (per CDM) of chaser
    "c_time_lastob_start",
    # start of the time in days of the last accepted observation used in the orbit determination of chaser
    "c_time_lastob_end",
    # end of the time interval in days of the last accepted observation used in the orbit determination of chaser
    "mahalanobis_distance",  # The distance between the chaser and target
    "miss_distance",  # relative position between chaser & target at tca [m
    "time_to_tca",  # Time interval between CDM creation and time-of-closest approach [days]
    "t_cndot_r",
    # covariance; correlation of normal (cross-track) velocity vs radial position of chaser
    "c_cr_area_over_mass",
    # solar radiation coefficient . A/m (ballistic coefficient equivalent) of chaser
    "max_risk_estimate",  # maximum collision probability obtained by scaling combined covariance
    "c_span",  # size used by the collision risk computation algorithm of chaser [m]
    "max_risk_scaling",  # scaling factor used to compute maximum collision probability
    "t_rcs_estimate",  # radar cross-sectional area [m2m2] of target
    "c_sigma_t",
    # covariance; transverse (along-track) position standard deviation (sigma) of chaser [m]
    "c_obs_available",  # number of observations available for orbit determination (per CDM),
    "risk",
]

In [ ]:
# Load and data and filter it
df = load_data()
df_filtered = filter_data(df)

# Remove rows with missing values
df_filtered.dropna(axis=0, how="any", inplace=True)

# Label encode the categorical column "c_object_type"
label_encode_column(df_filtered, "c_object_type")
# Call the function to get the processed DataFrame
df_processed = df_filtered[selected_features]

# Separate features and target variable
X = df_processed.drop('risk', axis=1)
y = df_processed['risk']

# Split the data into training, testing, and validation sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

# Perform scaling after splitting the data
scaler = StandardScaler()

# Fit the scaler on the training set
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same scaler to the validation and test sets
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Display the shapes of the sets
print("Full Training set shape:", X_train_full.shape, y_train_full.shape)
print("Testing set shape:", X_test.shape, y_test.shape)
print("Training set shape:", X_train_scaled.shape, y_train.shape)
print("Validation set shape:", X_val_scaled.shape, y_val.shape)


## Model Selection

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Choose Models
models = {
    # 'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor()
}

# Hyperparameter Tuning
param_grid = {
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
}

best_models = {}

for model_name, model in models.items():
    if model_name in param_grid:
        # Use GridSearchCV for hyperparameter tuning
        grid_search = GridSearchCV(model, param_grid[model_name], scoring='neg_mean_squared_error', cv=5)
        grid_search.fit(X_train_scaled, y_train)
        best_models[model_name] = grid_search.best_estimator_
    else:
        # For models without hyperparameters, use the default configuration
        best_models[model_name] = model.fit(X_train_scaled, y_train)


## Model Training:

In [ ]:
# Train Models
for model_name, model in best_models.items():
    model.fit(X_train_scaled, y_train)

# Evaluate Performance
from sklearn.metrics import mean_squared_error

# Evaluate on training set
for model_name, model in best_models.items():
    y_pred_train = model.predict(X_train_scaled)
    mse_train = mean_squared_error(y_train, y_pred_train)
    print(f'{model_name} - MSE on training set: {mse_train}')

# Evaluate on validation set
for model_name, model in best_models.items():
    y_pred_val = model.predict(X_val_scaled)
    mse_val = mean_squared_error(y_val, y_pred_val)
    print(f'{model_name} - MSE on validation set: {mse_val}')


In [ ]:
from sklearn.metrics import mean_absolute_error

# Assuming y_pred is your predicted values and y_true is the true target values
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# Calculate MAE for training set
mae_train = mean_absolute_error(y_train, y_pred_train)

# Calculate MAE for validation set
mae_val = mean_absolute_error(y_val, y_pred_val)

# Print the MAE values
print(f"MAE on training set: {mae_train}")
print(f"MAE on validation set: {mae_val}")


In [ ]:
from sklearn.metrics import r2_score

# Calculate R-squared for training set
r2_train = r2_score(y_train, y_pred_train)

# Calculate R-squared for validation set
r2_val = r2_score(y_val, y_pred_val)

# Print the R-squared values
print(f"R-squared on training set: {r2_train}")
print(f"R-squared on validation set: {r2_val}")
print("hi")

## Model Comparison:

## Fine-Tuning